option 1

getting the data. analyze diabetes data ..diff states, dif ages, pick a feature( a column to compair) two states , two zip code,... any two group descriptive statistics cleaning one or more visualization brief conclusion statement. health care industry is very codified. The coding then comes into play with this data set....need a data dictionary. renaming some columns that actual humans understand.

option 2

he didnt give the data... you have to get it yourself. figure out a way to import it... easiest is to download to your desktop. use read_csv..use the web link.... drive... who knows

make sure you look at your bias and the datas bias and dictate it in your summary. check out plotly can create interactive visualizations and cufflinks

will use seaborn can export visualizations just using code geopandas..location data... uber lyft... all use the geopandas library. SEABORN lecture 7

In [10]:
# coding: utf-8

NameError: ignored

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
#https://data.cityofnewyork.us/api/odata/v4/h9gi-nx95

SyntaxError: ignored

In [8]:
# # Function to load the complete dataset and perform the data manipulations in-house
# def load_data_complete():
#     cur_dir = os.path.dirname()

In [2]:
# Read the csv
mvc_c = pd.read_csv('/content/drive/MyDrive/PredictiveAnalytics/101/data/Motor_Vehicle_Collisions_-_Crashes.csv')


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# Converting the date column to its specified data type
mvc_c['DATE'] = pd.to_datetime(mvc_c.DATE)

AttributeError: ignored

In [ ]:
# Creating three new columns for easy querying
mvc_c['YEAR'] = mvc_c.DATE.dt.year
mvc_c['MONTH'] = mvc_c.DATE.dt.month
mvc_c['DAY'] = mvc_c.DATE.dt.day

In [ ]:
# Considering columns beneficial for visualizing on carto
mvc_c = mvc_c[['DATE', 'TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE',
               'CONTRIBUTING FACTOR VEHICLE 1', 'NUMBER OF PEDESTRIANS INJURED', 
               'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', 
               'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED']]

In [ ]:
# We notice that zip codes are of object datatype instead of the required integer data type
mvc_c = mvc_c[pd.notnull(mvc_c['ZIP CODE'])]

In [ ]:
# Converting zip codes datatype to numeric
mvc_c['ZIP CODE'] = pd.to_numeric(mvc_c['ZIP CODE'], errors='coerce')

In [ ]:
# Removing outliers and unrecorded zip code values from the data( i.e. zero values and those outside the NTC city limits)
mvc_c = mvc_c[(mvc_c['LATITUDE'] > 0.1) & (mvc_c['LATITUDE'] < 41)]
mvc_c = mvc_c.drop(mvc_c[mvc_c['CONTRIBUTING FACTOR VEHICLE 1']=='Unspecified'].index)

In [ ]:
# Merging the dataframe for carto with above zip counts
mvc_c = pd.merge(mvc_c, mvc_c['ZIP CODE'].value_counts().to_frame().reset_index().rename(columns={"ZIP CODE": "ZIP COUNTS"}), how='left', left_on='ZIP CODE', right_on='index').drop(columns='index')

In [ ]:
mvc_c = mvc_c.rename(columns={"CONTRIBUTING FACTOR VEHICLE 1": "CONTRIBUTING FACTOR"})
return mvc_c.sort_values('DATE', ascending=False)[:499999]
#.to_csv('dropped_500k_wCounts.csv', index=False)

In [ ]:
# Function to load the processed data obtained from running the above function 'load_data_complete' on the dataset
def load_data_concise():
    cur_dir = os.path.dirname('__file__')
    return pd.read_csv(os.path.join(cur_dir, "dropped_500k_wCounts.csv"))

In [ ]:
# Returns Viz 1: Total accidents per year for each borough
def time_pivot(data):
    tdf = data[['YEAR', 'BOROUGH']].groupby(['YEAR', 'BOROUGH']).size().rename('Count').reset_index()
    return pd.pivot_table(tdf, columns='YEAR', index='BOROUGH')

In [ ]:
# Returns Viz 2:  Injury/Fatality of a specific mode per year for each borough
def mode_pivot(data, mode):
    mdf = data[['DATE', 'BOROUGH', 'YEAR',
                   'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
                   'NUMBER OF CYCLIST INJURED'    , 'NUMBER OF CYCLIST KILLED',
                   'NUMBER OF MOTORIST INJURED'   , 'NUMBER OF MOTORIST KILLED']].groupby(['YEAR', 'BOROUGH']).sum()
    return pd.pivot_table(mdf, columns='YEAR', index='BOROUGH', values=mode)

In [ ]:
# Returns Viz 3:  Collision due to a specific Contriburing Factor per year for each borough
def factor_pivot(data, factor):
    top20_factors = ['Driver Inattention--Distraction',
                     'Failure to Yield Right-of-Way',
                     'Fatigued--Drowsy',
                     'Backing Unsafely',
                     'Other Vehicular',
                     'Following Too Closely',
                     'Turning Improperly',
                     'Lost Consciousness',
                     'Passing or Lane Usage Improper',
                     'Traffic Control Disregarded',
                     'Driver Inexperience',
                     'Prescription Medication',
                     'Unsafe Lane Changing',
                     'Pavement Slippery',
                     'Outside Car Distraction',
                     'Alcohol Involvement',
                     'Physical Disability',
                     'Oversized Vehicle',
                     'Reaction to Other Uninvolved Vehicle',
                     'Unsafe Speed']

In [ ]:
fdf = data[['BOROUGH', 'YEAR', 'CONTRIBUTING FACTOR']].groupby(['YEAR', 'BOROUGH', 'CONTRIBUTING FACTOR']).size().rename('Count').reset_index()
fdf = fdf[fdf['CONTRIBUTING FACTOR'].isin(top20_factors)]
return pd.pivot_table(fdf, columns='YEAR', index=['BOROUGH', 'CONTRIBUTING FACTOR']).xs(factor, level=1)#.to_csv('viz3_factor_pivot_'+factor+'.csv')